# Updated Code

In [146]:
def find_prerequisites(course_code, classes):
    def recursive_helper(course_code, classes):
        # Check if the course has prerequisites
        if 'prerequisites' in classes[course_code]:
            # Get the raw string of prerequisites for the current course
            prerequisites_str = classes[course_code]['prerequisites']

            # Check if prerequisites are not empty
            if prerequisites_str.strip() != '':
                # Split the raw string into individual prerequisites
                prerequisites_list = [prerequisite.strip() for prerequisite in prerequisites_str.split('or')]

                # Initialize the list to store the current branch of prerequisites
                current_branch = []

                # Process each prerequisite separately
                for prerequisite in prerequisites_list:
                    # Recursively find prerequisites for each prerequisite and add to the current branch
                    current_branch.extend(recursive_helper(prerequisite, classes))

                return current_branch

        # If the course has no prerequisites or empty prerequisites, return an empty list
        return []

    # Call the recursive helper function to get the overall list of courses
    return recursive_helper(course_code, classes)

In [147]:
# Example usage:
classes = {
    'STAT 400': {'prerequisites': 'MATH 141'},
    'MATH 141': {'prerequisites': 'MATH 140'},
    'MATH 140': {'prerequisites': 'MATH 108 or MATH 115'},
    'MATH 108': {'prerequisites': 'MATH 107'},
    'MATH 107': {'prerequisites': ''},
    'MATH 115': {'prerequisites': 'STAT 101 or MATH 100'},
    'STAT 101': {'prerequisites': ''},
    'MATH 100': {'prerequisites': ''}
}

In [148]:
course_code = 'STAT 400'
prerequisites = find_prerequisites(course_code, classes)

In [149]:
print(f"Prerequisites for {course_code}: {prerequisites}")

Prerequisites for STAT 400: []


In [144]:
# Example usage:
classes = {
    'STAT 400': {'prerequisites': 'MATH 141'},
    'MATH 141': {'prerequisites': 'MATH 140'},
    'MATH 140': {'prerequisites': 'MATH 108 or MATH 115'},
    'MATH 108': {'prerequisites': 'MATH 107'},
    'MATH 107': {'prerequisites': ''},
    'MATH 115': {'prerequisites': 'STAT 101 or MATH 100'},
    'STAT 101': {'prerequisites': ''},
    'MATH 100': {'prerequisites': ''}
}

In [145]:
course_code = 'STAT 400'
prerequisites = find_prerequisites(course_code, classes)

KeyError: ''

In [ ]:
print(f"Prerequisites for {course_code}: {prerequisites}")

In [50]:
def find_prerequisites(course_name, classes):
    # Check if the course has prerequisites 
    if 'prerequisites' in classes[course_name]:
        if classes[course_name] == '':
            # If prerequisites are empty, return nothing
            return ''
        else:
            # Get the raw string of prerequisites for the current course
            prerequisites_str = classes[course_name]['prerequisites']

            # Split the raw string into individual prerequisites
            prerequisites_list = [prerequisite.strip() for prerequisite in prerequisites_str.split('or')]

            # Initialize the list to store the complete set of prerequisites
            all_prerequisites = []

            # Process each branch separately
            for prerequisite in prerequisites_list:
                # Recursively find prerequisites for each branch
                branch_prerequisites = find_prerequisites(prerequisite, classes)
                # Add the current branch to the overall list
                all_prerequisites.append(branch_prerequisites)

            return all_prerequisites
    else:
        # If the course has no prerequisites, return an empty list
        return []

In [51]:
find_prerequisites('STAT 400', classes)

KeyError: ''

In [ ]:
# Example usage:
classes = {
    'STAT 400': {'prerequisites': 'MATH 141'},
    'MATH 141': {'prerequisites': 'MATH 140'},
    'MATH 140': {'prerequisites': 'MATH 108 or MATH 115'},
    'MATH 108': {'prerequisites': 'MATH 107'},
    'MATH 107': {'prerequisites': ''},
    'MATH 115': {'prerequisites': ''}
}

course_code = 'STAT 400'
prerequisites = find_prerequisites(course_code, classes)

print(f"Prerequisites for {course_code}: {prerequisites}")